In [1]:
import pybamm;import pandas as pd   ;import numpy as np;import os;import matplotlib.pyplot as plt;import os;#import imageio
from scipy.io import savemat,loadmat;from pybamm import constants,exp,sqrt;
import matplotlib as mpl; 
fs=17; # or we can set import matplotlib.pyplot as plt then say 'mpl.rc...'
import openpyxl
import traceback
import multiprocessing
font = {'family' : 'DejaVu Sans','size'   : fs}
mpl.rc('font', **font)

In [2]:
V_max = 4.2;        V_min = 2.5;
exp  = pybamm.Experiment(
    [ 
        (f"Discharge at 1 C until {V_min} V", 
        f"Charge at 1 C until {V_max} V",  # (5 minute period)
        f"Hold at {V_max} V until C/20"),] * 1 )  

In [3]:
model  = pybamm.lithium_ion.DFN(options={
    "SEI":"interstitial-diffusion limited",            
    "SEI film resistance":"distributed",          
    "SEI porosity change":"true",   
    "electrolyte conductivity": "full"  ,}) 

In [4]:
para=pybamm.ParameterValues("Chen2020")
para.update({'Inner SEI lithium interstitial diffusivity [m2.s-1]':3e-19})

In [5]:
var_pts = {
    "x_n": 10,  # negative electrode
    "x_s": 5,  # separator 
    "x_p": 10,  # positive electrode
    "r_n": 30,  # negative particle
    "r_p": 20,  # positive particle
}

sim    = pybamm.Simulation(
    model, experiment = exp,
    parameter_values = para,
    solver = pybamm.CasadiSolver(return_solution_if_failed_early=True),
    var_pts=var_pts,)       
sol = sim.solve()

KeyError: "'EC concentration [mol.m-3]' not found. Best matches are ['SEI concentration [mol.m-3]', 'Outer SEI concentration [mol.m-3]', 'Inner SEI concentration [mol.m-3]']"

In [ ]:
# compare int - Dec
Str = ["Int-Single-H_Dec",]
font = {'family' : 'DejaVu Sans','size'   : 12}
mpl.rc('font', **font)
var_elely = [
    "Terminal voltage [V]",
    "Negative electrode surface potential difference [V]",
    "Negative electrode potential [V]",
    "Negative electrolyte potential [V]",
]
quick_plot = pybamm.QuickPlot(
    sol, var_elely,
    Str,variable_limits='tight',
    time_unit='hours',
    spatial_unit='mm',     #  (“m”, “mm”, or “um”)
    n_rows=2) #figsize = (18,12),
quick_plot.dynamic_plot()